## Installing necessary libraries

In [ ]:
! /opt/conda/bin/python3.7 -m pip install -q --upgrade pip
! pip install -q timm catalyst iterative-stratification
! pip install -q --upgrade wandb
! pip install -q pytorch-gradcam

We are going to use `wandb` for tracking our model's performance. If you don't have a `wandb` account, go to [this](wandb.ai) link, create an account using either google or github account. Then go to `wandb.ai/[your_username]` -> `Create New Project`. Give a cute little name to your project. Open your project page. You'll find some line like this:

`wandb login e1da498db2dd649a76a04c6e4743e5a4f95a2ae0`

Copy and paste this line to the next cell.

In [ ]:
! wandb login e1da498db2dd649a76a04c6e4743e5a4f95a2ae0

# Config
This section contains configuration parameters for my classification pipeline.

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import cv2
import pandas as pd
import albumentations as A
from albumentations.augmentations.transforms import Equalize, Posterize, Downscale
from albumentations import (
    PadIfNeeded, HorizontalFlip, VerticalFlip, CenterCrop,    
    RandomCrop, Resize, Crop, Compose, HueSaturationValue,
    Transpose, RandomRotate90, ElasticTransform, GridDistortion, 
    OpticalDistortion, RandomSizedCrop, Resize, CenterCrop,
    VerticalFlip, HorizontalFlip, OneOf, CLAHE, Normalize,
    RandomBrightnessContrast, Cutout, RandomGamma, ShiftScaleRotate ,
    GaussNoise, Blur, MotionBlur, GaussianBlur, 
)

SEED = 24
n_epochs = 25
device = 'cuda:0'
data_dir = '../input/ranzcr-clip-catheter-line-classification'
loss_thr = 1e6
img_path = '../input/ranzcr-clip-1024-resized/resized_1024'
df = pd.read_csv(f'{data_dir}/train.csv')
df['path'] = df['StudyInstanceUID'].map(lambda x: f"{img_path}/{x}.jpg")
encoder_model = 'resnest50_fast_1s1x64d'
fold = 0
model_name= f'Resnest50_fold{fold}' # Will come up with a better name later
model_dir = 'model_dir'
history_dir = 'history_dir'
load_model = False
img_dim = 320
batch_size = 40
accum_step = 1
learning_rate = 7.50e-3
num_workers = 4
mixed_precision = True
patience = 3
balanced_sampler = False
train_aug = A.Compose([A.CenterCrop(p=0.3, height=int(0.8*img_dim), width=int(0.8*img_dim)),
A.augmentations.transforms.RandomCrop(int(0.8*img_dim), int(0.8*img_dim), p=0.3),
A.augmentations.transforms.Rotate(limit=30, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=False, p=0.5),
A.augmentations.transforms.Resize(img_dim, img_dim, interpolation=1, always_apply=True, p=0.6),
Cutout(num_holes=8, max_h_size=20, max_w_size=20, fill_value=0, always_apply=False, p=0.2),
A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, brightness_by_max=True, always_apply=False, p=0.3),
A.augmentations.transforms.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=20, always_apply=False, p=0.4),
# A.HorizontalFlip(p=0.5),
A.VerticalFlip(p=0.5),                    
OneOf([
        GaussNoise(var_limit=0.1),
        Blur(),
        GaussianBlur(blur_limit=3),
        # RandomGamma(p=0.7),
        ], p=0.3),
A.HorizontalFlip(p=0.3), Normalize(always_apply=True)],)
val_aug = Compose([Normalize(always_apply=True)])

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
fig = plt.figure(figsize = (15,20))
ax = fig.gca()
df.hist(ax = ax)

## Fixing Seed

In [ ]:
import os
import random
import numpy as np
import torch

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(SEED)

## Data Stratification

In [ ]:
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold 
mskf = MultilabelStratifiedKFold(n_splits=5, random_state=SEED)
X = df['path']
y = df.iloc[:, 1:12]
train_idx = []
val_idx = []

df['fold'] = np.nan

#split data
for i, (_, test_index) in enumerate(mskf.split(X, y)):
    df.loc[test_index, 'fold'] = i
    
df['fold'] = df['fold'].astype('int')

valid_df = df[df['fold']==fold]
train_df = df[df['fold']!=fold]

## Dataset

In [ ]:
from torch.utils.data import Dataset,DataLoader

class CatheterDataset(Dataset):
    def __init__(self, df, dim=256, transforms=None):
        super().__init__()
        self.image_ids = df.path.tolist()
        try:
            self.labels = np.array(df.iloc[:, 1:12])
        except:
            self.labels = None
        self.transforms = transforms
        self.dim = dim
        
    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        image = cv2.imread(image_id, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (self.dim, self.dim))
        
        if self.transforms is not None:
            aug = self.transforms(image=image)
            image = aug['image'].reshape(self.dim, self.dim, 3).transpose(2, 0, 1)
        else:
            image = image.reshape(self.dim, self.dim, 3).transpose(2, 0, 1)
        if self.labels is not None:
            target = self.labels[idx]
            return image_id, image, target
        else:
            return image_id, image

    def __len__(self):
        return len(self.image_ids)

    def get_labels(self):
        return list(self.labels)

## Model

### Generalized Mean Pooling

I'm using `Resnest` model for training and replacing the last `global_pool` layer with Generalized Mean Pooling(GeM) layer from [here](https://www.kaggle.com/c/aptos2019-blindness-detection/discussion/108065).  

In [ ]:
from torch import nn

def gem(x, p=3, eps=1e-6):
    return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)

class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM,self).__init__()
        self.p = Parameter(torch.ones(1)*p)
        self.eps = eps
    def forward(self, x):
        return gem(x, p=self.p, eps=self.eps)       
    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + ', ' + 'eps=' + str(self.eps) + ')'

In [ ]:
from torch import nn
from torch.nn import *
from torch.nn import functional as F
from torchvision import models
import timm

class Resne_t(nn.Module):

    def __init__(self, model_name):
        super().__init__()
        try:
            self.backbone = timm.create_model(model_name, pretrained=True)
        except:
            self.backbone = torch.hub.load('zhanghang1989/ResNeSt', model_name, pretrained=True)
        self.in_features = self.backbone.fc.in_features
        self.output = nn.Sequential(nn.Linear(self.in_features, 128), nn.Linear(128, 11))
        self.backbone.global_pool = GeM()

    def forward(self, x):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        try:
            x = self.backbone.act1(x)
        except:
            x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)
        x = self.backbone.global_pool(x)
        x = x.view(x.size(0), -1)
        x = self.output(x)
        return x

# Loss function

I'm going to use `Focal Cosine Loss` in this notebook. The implementation is borrowed from [here](https://github.com/byeongjokim/VIPriors-Image-Classification-Challenge/blob/332e04fd3e82b20d312128bad302a9081f5c37ce/timm/loss/cosine.py) and modified for multi-label classification.

In [ ]:
class FocalCosineLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, xent=.1, reduction="mean"):
        super(FocalCosineLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

        self.xent = xent
        self.reduction = reduction
        
        self.y = torch.Tensor([1]).cuda()
        
    def forward(self, input, target):
        cosine_loss = F.cosine_embedding_loss(input, target, self.y, reduction=self.reduction)
        cent_loss = nn.BCEWithLogitsLoss()(input, target)
        pt = torch.exp(-cent_loss)
        focal_loss = self.alpha * (1-pt)**self.gamma * cent_loss

        if self.reduction == "mean":
            focal_loss = torch.mean(focal_loss)
        
        return cosine_loss + self.xent * focal_loss

# Utils

After running this cell, you will get a link titled `Run Page` that looks like this: `https://wandb.ai/[user_name]/[project_name]/runs/********`. If you commit this notebook, then you won't be able to see this link. In that case, go to your project page i.e., `https://wandb.ai/[user_name]/[project_name]/` and find the running project (the one with a green knob). Your Training logs should be available there.

In [ ]:
import logging
logging.basicConfig(level=logging.ERROR)
import wandb
from functools import partial
from collections import Counter
import gc
import time
import pandas as pd
from torch import optim
from catalyst.data.sampler import BalanceClassSampler

wandb.init(project="catheter")
wandb.run.name= model_name

m_p = mixed_precision
if m_p:
  scaler = torch.cuda.amp.GradScaler() 

np.random.seed(SEED)

train_ds = CatheterDataset(train_df, img_dim, train_aug)
if balanced_sampler:
  print('Using Balanced Sampler....')
  train_loader = torch.utils.data.DataLoader(train_ds,batch_size=batch_size, sampler=BalanceClassSampler(labels=train_ds.get_labels(), mode="upsampling"), shuffle=False, num_workers=4)
else:
  train_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)

val_ds = CatheterDataset(valid_df, img_dim, val_aug)
valid_loader = torch.utils.data.DataLoader(
val_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

os.makedirs(model_dir, exist_ok=True)
os.makedirs(history_dir, exist_ok=True)

result = pd.DataFrame(columns=['name', 'prediction', 'label', 'difference'])
if os.path.exists(f'{history_dir}/history_{model_name}_{img_dim}.csv'):
    history = pd.read_csv(f'{history_dir}/history_{model_name}_{img_dim}.csv')
else:
    history = pd.DataFrame(columns=['train_loss','train_time','val_loss','val_roc_auc', 'val_time'])

model = Resne_t(encoder_model).to(device)
wandb.watch(model)
# criterion = nn.BCEWithLogitsLoss()
criterion = FocalCosineLoss(reduction='sum')

In [ ]:
def save_model(valid_loss, valid_roc, best_valid_loss, best_valid_roc, best_state, savepath):
    if valid_loss<best_valid_loss:
        print(f'Validation loss has decreased from:  {best_valid_loss:.4f} to: {valid_loss:.4f}. Saving checkpoint')
        torch.save(best_state, savepath+'_loss.pth')
        best_valid_loss = valid_loss
    if valid_roc>best_valid_roc:
        print(f'Validation ROC_AUC score has increased from:  {best_valid_roc:.4f} to: {valid_roc:.4f}. Saving checkpoint')
        torch.save(best_state, savepath + '_roc_auc.pth')
        best_valid_roc = valid_roc
    else:
        torch.save(best_state, savepath + '_last.pth')
    return best_valid_loss, best_valid_roc

# ROC_AUC

In [ ]:
from sklearn.metrics import roc_auc_score
def ROC(predictions, labels):
    tmp_roc = []
    for i in range(11):
        tmp_roc.append(roc_auc_score(np.array(labels)[:, i], np.array(predictions)[:, i], ))
    return np.mean(tmp_roc)

# Training

In [ ]:
def train_val(epoch, dataloader, optimizer, pretrained=None, train=True, mode='train', record=True):
    global m_p
    global result
    global batch_size
    global accum_step
    t1 = time.time()
    running_loss = 0
    epoch_samples = 0
    pred = []
    lab = []
    if pretrained:
        model.load_state_dict(pretrained)
    if train:
        model.train()
        print("Initiating train phase ...")
    else:
        model.eval()
        print("Initiating val phase ...")
    for idx, (_, img, labels) in enumerate(dataloader):
        with torch.set_grad_enabled(train):
            img = img.to(device, dtype=torch.float32)
            labels = labels.to(device, dtype=torch.float32)
            epoch_samples += len(img)
            optimizer.zero_grad()
            with torch.cuda.amp.autocast(m_p):
                if m_p:
                    img = img.half()
                else:
                    img = img.float()
                outputs = model(img)

                loss = criterion(outputs, labels).sum()
                running_loss += loss.item()*len(img)
                loss = loss/accum_step
      
                if train:
                     if m_p:
                         scaler.scale(loss).backward()
                         if (idx+1) % accum_step == 0:
                             scaler.step(optimizer)
                             scaler.update() 
                             optimizer.zero_grad()
                     else:
                         loss.backward()
                         if (idx+1) % accum_step == 0:
                             optimizer.step()
                             optimizer.zero_grad()

        elapsed = int(time.time() - t1)
        eta = int(elapsed / (idx+1) * (len(dataloader)-(idx+1)))
        pred.extend(torch.sigmoid(outputs).detach().cpu().numpy())
        lab.extend(labels.cpu().numpy())

        if train:
            msg = f"Epoch: {epoch} Progress: [{idx}/{len(dataloader)}] loss: {(running_loss/epoch_samples):.4f} Time: {elapsed}s ETA: {eta} s"
        else:
            msg = f'Epoch {epoch} Progress: [{idx}/{len(dataloader)}] loss: {(running_loss/epoch_samples):.4f} Time: {elapsed}s ETA: {eta} s'
        wandb.log({"Train Loss": running_loss/epoch_samples, "Epoch":epoch})
        print(msg, end= '\r')
    roc = ROC(np.array(pred), np.array(lab))
    history.loc[epoch, f'{mode}_loss'] = running_loss/epoch_samples
    history.loc[epoch, f'{mode}_time'] = elapsed
    if mode=='val' or mode=='test':
        lr_reduce_scheduler.step(roc)
        msg = f'{mode} Loss: {running_loss/epoch_samples:.4f} \n {mode} ROC_AUC: {roc:.4f}'
        print(msg)
        wandb.log({f"{mode} Loss": running_loss/epoch_samples, f"{mode} ROC_AUC":roc, "Epoch":epoch})
        history.loc[epoch, f'{mode}_loss'] = running_loss/epoch_samples
        history.loc[epoch, f'{mode}_roc_auc'] = roc
        # NaN check
        if running_loss/epoch_samples > loss_thr or running_loss!=running_loss:
            print('\033[91mMixed Precision\033[0m rendering nan value. Forcing \033[91mMixed Precision\033[0m to be False ...')
            m_p = False
            batch_size = batch_size//2
            accum_step = accum_step*2
            print('Loading last best model ...')
            tmp = torch.load(os.path.join(model_dir, model_name+'_loss.pth'))
            model.load_state_dict(tmp['model'])
            optimizer.load_state_dict(tmp['optim'])
            lr_reduce_scheduler.load_state_dict(tmp['scheduler'])
            del tmp
            
        if record:
            history.to_csv(f'{history_dir}/history_{model_name}_{img_dim}.csv', index=False)
        return running_loss/epoch_samples, roc


plist = [ 
        {'params': model.backbone.parameters(),  'lr': learning_rate/50},
        {'params': model.output.parameters(),  'lr': learning_rate}
    ]
optimizer = optim.Adam(plist, lr=learning_rate)
lr_reduce_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=patience, verbose=True, threshold=1e-4, threshold_mode='rel', cooldown=0, min_lr=1e-7, eps=1e-08)

In [ ]:
def main():
  prev_epoch_num = 0
  best_valid_loss = np.inf
  best_valid_roc = 0.0

  if load_model:
    tmp = torch.load(os.path.join(model_dir, model_name+'_roc_auc.pth'))
    model.load_state_dict(tmp['model'])
    optimizer.load_state_dict(tmp['optim'])
    lr_reduce_scheduler.load_state_dict(tmp['scheduler'])
    scaler.load_state_dict(tmp['scaler'])
    prev_epoch_num = tmp['epoch']
    best_valid_loss = tmp['best_loss']
    best_valid_loss, best_valid_roc = train_val(prev_epoch_num+1, valid_loader, optimizer=optimizer, train=False, mode='val')
    del tmp
    print('Model Loaded!')
  
  for epoch in range(prev_epoch_num, n_epochs):
    torch.cuda.empty_cache()
    print(gc.collect())

    train_val(epoch, train_loader, optimizer=optimizer, train=True, mode='train')
    valid_loss, valid_roc = train_val(epoch, valid_loader, optimizer=optimizer, train=False, mode='val')
    print("#"*20)
    print(f"Epoch {epoch} Report:")
    print(f"Validation Loss: {valid_loss :.4f} Validation ROC_AUC: {valid_roc :.4f}")
    best_state = {'model': model.state_dict(), 'optim': optimizer.state_dict(), 'scheduler':lr_reduce_scheduler.state_dict(), 
          'scaler': scaler.state_dict(),
    'best_loss':valid_loss, 'best_acc':valid_roc, 'epoch':epoch}
    best_valid_loss, best_valid_roc = save_model(valid_loss, valid_roc, best_valid_loss, best_valid_roc, best_state, os.path.join(model_dir, model_name))
    print("#"*20)
   
if __name__== '__main__':
  main()